dataset preperation file
It contains several parts, each performs another step in perpering the dataset to be able to run the SVM on it and get the new direction
This file was made specificaly for the "Shoulder pain dataset" and will need to be modified to fit any other dataset
the "Shoulder pain dataset" structure: 40 subjects, each has few videos saved as "frames". 
each frame has a txt file containing the facs score for that frmae.
the SVM is designed to take inputs in the form of 2 vectors: a vector of the latant encoding for each image and a vector with corresponding scores.
the steps:
1. create json file with "patient_name", "video_name","frame" and pspi_score" keys
2. add "facs score" key
3. add "pain" key
4. split to train validation and test and add statistics
5. copy the images to 3 folders based on the split
6. after the embedding of the images in the folders, craete the 2 output vectors of lataent codes and score


section 1: This section reads all the txt files and merge them into one json file containing "patient_name", "video_name","frame" and pspi_score"

In [17]:
import os
import json

parent_directory = "Frame_Labels/PSPI"

data_list = []  # List to store data for all text files

for directory in os.listdir(parent_directory):
    current_directory = os.path.join(parent_directory, directory)
    if os.path.isdir(current_directory):
        for root, directories, files in os.walk(current_directory):
            for file_name in files:
                if file_name.endswith('.txt'):  # Process only .txt files
                    file_path = os.path.join(root, file_name)
                    with open(file_path, 'r') as file:
                        content = file.read().strip()  # Read content and remove leading/trailing whitespaces
                        try:
                            numeric_value = float(content)  # Convert the content to a float
                            numeric_value_int = int(numeric_value)  # Convert float to integer
                        except ValueError:
                            print(f"Error converting content from file: {file_path}")
                            continue  # Skip this file if conversion fails

                        video_name = os.path.basename(root)  # Get the name of the directory containing the text files
                        frame_name = os.path.splitext(file_name)[0]  # Remove the file extension ".txt"

                        frame_data = {
                            "patient_name": directory,  # Set the patient name as the directory name
                            "video_name": video_name,
                            "frame": frame_name,
                            "pspi_score": numeric_value_int  # Store the converted integer value
                        }
                        data_list.append(frame_data)

# Save combined data to a single JSON file
output_file_name = "pspi.json"  # Name for the merged JSON file
with open(output_file_name, 'w') as output_file:
    json.dump(data_list, output_file, indent=4)

print(f"Data saved to {output_file_name}")


Data saved to pspi.json


section 2: This section adds the "facs_score" to the json file

In [18]:
import os
import json

parent_directory = "Frame_Labels/FACS"  # Replace this with your parent directory path
json_path = "pspi.json"

if os.path.exists(json_path) and os.path.isfile(json_path):
    with open(json_path, 'r') as json_file:
        data = json.load(json_file)

        for patient_dir in os.listdir(parent_directory):
            patient_path = os.path.join(parent_directory, patient_dir)
            if os.path.isdir(patient_path):
                for video_dir in os.listdir(patient_path):
                    video_path = os.path.join(patient_path, video_dir)
                    if os.path.isdir(video_path):
                        for txt_file in os.listdir(video_path):
                            if txt_file.endswith('.txt'):
                                txt_file_path = os.path.join(video_path, txt_file)
                                frame_name = os.path.splitext(txt_file)[0]

                                # Search for the corresponding frame in the JSON data
                                for item in data:
                                    if item['patient_name'] == patient_dir and item['video_name'] == video_dir and item['frame'] == frame_name:
                                        with open(txt_file_path, 'r') as txt_file_content:
                                            content = txt_file_content.readlines()
                                            numbers_list = []

                                            for line in content:
                                                numbers = [int(float(num)) for num in line.split()]
                                                numbers_list.append(numbers)

                                            item['facs_score'] = numbers_list  # Update 'Facs Score' as a list of lists

    # Save updated data back to the JSON file
    with open(json_path, 'w') as updated_json_file:
        json.dump(data, updated_json_file, indent=4)
        print(f"Facs Scores updated in {json_path}")
else:
    print(f"JSON file not found at {json_path}")


Facs Scores updated in pspi.json


section 3: This section add a "pain" key to each frame in the json file, based on the pspi score 

In [4]:
import os
import json

parent_directory = "Frame_Labels/FACS"  # Replace this with your parent directory path
json_path = "pspi.json"
output_json_path  = "Frame_Labels.json"

if os.path.exists(json_path) and os.path.isfile(json_path):
    with open(json_path, 'r') as json_file:
        data = json.load(json_file)

for frame in data:
    frame['pain'] = 1 if frame['pspi_score'] > 0 else 0

with open(output_json_path, 'w') as output_json_file:
    json.dump(data, output_json_file, indent=4)
    print(f"Datasets with 'pain' score saved to {output_json_path}")


Datasets with statistics, data, and 'pain' saved to Frame_Labels.json


section 4: Split Dataset into training 60% validaion 20% and test 20% , adds ststistics section and save in a new json file named "datased.json"

In [2]:
import os
import json
import random
from collections import defaultdict
from collections import deque

input_json_path = "Frame_Labels.json"  # Replace this with the path to the existing JSON file
output_json_path = "dataset.json"  # Replace this with the path for the output JSON file
total_samples = 10000 # Total number of selected samples
random.seed(42)  # Set seed for reproducibility

if os.path.exists(input_json_path) and os.path.isfile(input_json_path):
    with open(input_json_path, 'r') as json_file:
        data = json.load(json_file)

        # Group frames by patients
        patients = defaultdict(list)
        for item in data:
            patients[item['patient_name']].append(item)

        # Calculate statistics for the full dataset
        full_dataset_stats = {pspi: len([item for item in data if item['pspi_score'] == pspi]) for pspi in set(item['pspi_score'] for item in data)}

        # Selecting different patients for each subset
        all_patients = list(patients.keys())
        random.shuffle(all_patients)
        num_training_patients = int(len(all_patients) * 0.6)
        num_validation_patients = int(len(all_patients) * 0.2)
        training_patients = all_patients[:num_training_patients]
        validation_patients = all_patients[num_training_patients:num_training_patients + num_validation_patients]
        test_patients = all_patients[num_training_patients + num_validation_patients:]

        # Filter frames for each subset based on selected patients
        training_data = [frame for patient in training_patients for frame in patients[patient]]
        validation_data = [frame for patient in validation_patients for frame in patients[patient]]
        test_data = [frame for patient in test_patients for frame in patients[patient]]

        
        
        
        # Calculate the number of samples for each subset based on the proportion of total_samples
        num_training_samples = int(total_samples * 0.6)
        num_validation_samples = int(total_samples * 0.2)
        num_test_samples = total_samples - num_training_samples - num_validation_samples

        def filter_frames(dataset, num_samples):
            
            filtered_pain_stats = defaultdict(int)
            pain_frames = {0: deque(), 1: deque()}
            for frame in dataset:
                pain_frames[frame['pain']].append(frame)
            
            # Helper function to add frames based on 'pain' with preference to higher PSPI scores
            def add_frame(pain):
                if pain_frames[pain]:
                    return pain_frames[pain].pop()
                return None

            # Distribute frames with 'pain' 0 and 'pain' 1 equally among subsets
            filtered_dataset = []
            num_samples_per_pain = num_samples // 2  # Equal distribution of 'pain' 0 and 'pain' 1
            while len(filtered_dataset) < num_samples:
                # Add frames with 'pain' 0
                frame = add_frame(0)
                if frame:
                    filtered_dataset.append(frame)
                    filtered_pain_stats[frame['pain']] += 1

                # Add frames with 'pain' 1, favoring higher PSPI scores
                frame = add_frame(1)
                if frame:
                    filtered_dataset.append(frame)
                    filtered_pain_stats[frame['pain']] += 1

            print("Filtered Pain stats:", filtered_pain_stats)
            return filtered_dataset

        # Filter frames in each subset to achieve balanced PSPI scores
        training_data = filter_frames(training_data, num_training_samples)
        validation_data = filter_frames(validation_data, num_validation_samples)
        test_data = filter_frames(test_data, num_test_samples)

        # Assign 'sub set' key to each frame
        for frame in training_data:
            frame['subset'] = 'training'
        for frame in validation_data:
            frame['subset'] = 'validation'
        for frame in test_data:
            frame['subset'] = 'test'

        # Calculate statistics for each subset including patient names and total frames
        def calculate_statistics_with_patients(dataset, subset_name):
            stats = {pspi: len([item for item in dataset if item['pspi_score'] == pspi]) for pspi in set(item['pspi_score'] for item in dataset)}
            patients_list = list(set(item['patient_name'] for item in dataset))
            return {
                "subset": subset_name,
                "statistics": stats,
                "patients": patients_list,
                "num_patients": len(set(patients_list)),
                "total_frames": len(dataset)  # Total frames in the subset
            }

        training_stats = calculate_statistics_with_patients(training_data, "training")
        validation_stats = calculate_statistics_with_patients(validation_data, "validation")
        test_stats = calculate_statistics_with_patients(test_data, "test")

        # Add statistics for each subset
        datasets_stats = [
            {"subset": "full_dataset", "statistics": full_dataset_stats, "num_patients": len(all_patients), "total_frames": len(data)},
            training_stats,
            validation_stats,
            test_stats
        ]

        # Save datasets with statistics and actual data to a new JSON file
        datasets = {
            "statistics": datasets_stats,
            "training_data": training_data,
            "validation_data": validation_data,
            "test_data": test_data
        }

        with open(output_json_path, 'w') as output_json_file:
            json.dump(datasets, output_json_file, indent=4)
            print(f"Datasets with statistics and data saved to {output_json_path}")
else:
    print(f"JSON file not found at {input_json_path}")


Filtered Pain stats: defaultdict(<class 'int'>, {0: 3000, 1: 3000})
Filtered Pain stats: defaultdict(<class 'int'>, {0: 1000, 1: 1000})
Filtered Pain stats: defaultdict(<class 'int'>, {0: 1000, 1: 1000})
Datasets with statistics and data saved to dataset.json


section 5: Copy images to "training" "validation" and "test" folders

In [8]:
import json
import os
from shutil import copyfile

input_json_path = "dataset.json"  # Replace with the path to your dataset JSON file

if os.path.exists(input_json_path) and os.path.isfile(input_json_path):
    with open(input_json_path, 'r') as json_file:
        data = json.load(json_file)

        subsets = ["training", "validation", "test"]
        # Create folders for each subset if they don't exist
        #subset_folders = ["training_images", "validation_images", "test_images"]
        for subset_folder in subsets:
             os.makedirs(subset_folder, exist_ok=True)

        

        for subset in subsets:
            subset_data = data[f"{subset}_data"]

            # Iterate through frames in the subset
            for frame in subset_data:
                patient_name = frame["patient_name"]
                video_name = frame["video_name"]
                frame_number = frame["frame"][:-5]
                image_path = f"Images/{patient_name}/{video_name}/{frame_number}.png"
                subset_folder = subset



                # Check if the image exists and copy it to the corresponding subset folder
                if os.path.exists(image_path) and os.path.isfile(image_path):
                    destination_path = f"{subset_folder}/{frame_number}.png"
                    copyfile(image_path, destination_path)
                    print(f"Image {frame_number}.png copied to {subset_folder}")
                else:
                    print(f"Image {frame_number}.png not found in {image_path}")

    print("Images copied successfully.")
else:
    print(f"JSON file not found at {input_json_path}")


Image vw121t1aiaff135.png copied to training
Image vw121t1aaaff126.png copied to training
Image vw121t1aiaff134.png copied to training
Image vw121t1aaaff125.png copied to training
Image vw121t1aiaff133.png copied to training
Image vw121t1aaaff124.png copied to training
Image vw121t1aiaff132.png copied to training
Image vw121t1aaaff123.png copied to training
Image vw121t1aiaff131.png copied to training
Image vw121t1aaaff122.png copied to training
Image vw121t1aiaff130.png copied to training
Image vw121t1aaaff121.png copied to training
Image vw121t1aiaff129.png copied to training
Image vw121t1aaaff120.png copied to training
Image vw121t1aiaff128.png copied to training
Image vw121t1aaaff119.png copied to training
Image vw121t1aiaff127.png copied to training
Image vw121t1aaaff118.png copied to training
Image vw121t1aiaff126.png copied to training
Image vw121t1aaaff117.png copied to training
Image vw121t1aiaff125.png copied to training
Image vw121t1aaaff116.png copied to training
Image vw12

section 6: Take embedding directory and using the "dataset.json" file containing the scores for all images craete 2 vectors: 1 of latent codes and one of their scores.

In [32]:
import os
import json
import torch
import numpy as np

# Load latent codes and corresponding image names
latent_codes = []
image_names = []
embeddings_folder = "./training/embeddings"  # Replace with your folder path
output_json_path_latent = "./latent_codes.json"
output_json_path_scores = "./pain_scores.json"

for filename in os.listdir(embeddings_folder):
    if filename.endswith(".pt"):
        filepath = os.path.join(embeddings_folder, filename)
        w_pivot = torch.load(filepath)
        w_pivot = w_pivot[0,0,:]
        #print(f"w_pivot.shape = {w_pivot.shape}")
        latent_codes.append(w_pivot.cpu().detach().numpy())
        #print (f"w_pivot = {w_pivot.cpu().detach().numpy()}")
        image_names.append(filename[:-3] + "_facs")  # Add "_facs" to the image names

# Load the JSON file
json_file_path = "Frame_Labels.json"  # Replace with your JSON file path
if os.path.exists(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

        # Find corresponding "pain" scores for the image names
        pain_scores = []
        for image_name in image_names:
            for item in data:
                if item['frame'] == image_name:
                    pain_scores.append(item['pain'])
                    break  # Stop searching when a match is found

        # Convert lists to NumPy arrays
        latent_codes = np.array(latent_codes)
        pain_scores = np.array(pain_scores).reshape((len(pain_scores), 1))
        np.save("latent_codes.npy", latent_codes)
        np.save("pain_scores.npy", pain_scores)

        # Flatten the 4D array to 2D
        flattened_latent_codes = latent_codes.reshape(latent_codes.shape[0], -1)

        

        # Save NumPy arrays as CSV (text) files
        np.savetxt('latent_codes.txt', flattened_latent_codes, delimiter=',', fmt='%f')
        np.savetxt('pain_scores.txt', pain_scores, delimiter=',', fmt='%d')

        # Display the saved files
        print(f"Saved 'latent_codes.txt' and 'pain_scores.txt'.")
else:
    print(f"JSON file not found at {json_file_path}")





Saved 'latent_codes.txt' and 'pain_scores.txt'.
